In [25]:
# coordinate positions for scraping from different resolutions

fastRecordIconPositionsX = [1136, 1171, 1207, 1244, 1280, 1316, 1352, 1388, 1424, 
1424, 
1424, 1424, 1352, 1316, 1280, 1244, 1207, 1171, 1136, 
1136, 1171, 1207, 1244, 1280, 1316, 1352, 1388]

fastRecordIconPositionsY = [611, 611, 611, 611, 611, 611, 611, 611, 611, 
637, 
677, 677, 677, 677, 677, 677, 677, 677, 677,
637, 637, 637, 637, 637, 637, 637, 637]

'''920, 507
952, 507
992, 507
1032, 507
1062, 507
1102, 507
1132, 507
1132, 550
1102, 550
1062, 550
1032, 550
992, 550
952, 550
920, 550
920, 585
952, 585
992, 585
1032, 585
1062, 585
1102, 585
1132, 585
1132, 623
992, 623
952, 623
920, 623'''

list1 = [920, 952, 992, 1032, 1062, 1102, 1132, 1132, 1102, 1062, 1032, 992, 952, 920, 920, 952, 992, 1032, 1062, 1102, 1132, 1132, 992, 952, 920]
list2 = [507, 507, 507, 507, 507, 507, 507, 550, 550, 550, 550, 550, 550, 550, 585, 585, 585, 585, 585, 585, 585, 623, 623, 623, 623]

assert len(fastRecordIconPositionsX) == len(fastRecordIconPositionsY)

assert len(list1) == len(list2)

# List of Positions for each record icon x, y (at given GUI scale, resolution (4k), window size) 
_4kRecordIconPositionsX = [1842, 2274, 2274, 1842, 1842, 2274]
_4kRecordIconPositionsY = [1155, 1155, 1209, 1209, 1260, 1260]

_2kRecordIconPositionsX = [921, 1137, 1137, 921, 921, 1137]
_2kRecordIconPositionsY = [int(i) / 2 for i in _4kRecordIconPositionsY]

fastRecordIconPositionsX = [1136, 1171, 1207, 1244, 1280, 1316, 1352, 1388, 1424, 
1424, 
1424, 1388, 1352, 1316, 1280, 1244, 1207, 1171, 1136, 
1136, 1171, 1207, 1244, 1280, 1316, 1352, 1388]

fastRecordIconPositionsY = [611, 611, 611, 611, 611, 611, 611, 611, 611, 
637, 
677, 677, 677, 677, 677, 677, 677, 677, 677,
637, 637, 637, 637, 637, 637, 637, 637]

assert len(fastRecordIconPositionsX) == len(fastRecordIconPositionsY)

# create a list of tuples from the 2k lists
recordIconPositions = list(zip(_2kRecordIconPositionsX, _2kRecordIconPositionsY))

# create a list of tuples from the 4k lists
_4krecordIconPositions = list(zip(_4kRecordIconPositionsX, _4kRecordIconPositionsY))

# create an extended list with values intrapolated in between the given values
recordIconPositionsExtended = []
for i in range(0, 5):
    recordIconPositionsExtended.append(recordIconPositions[i])
    recordIconPositionsExtended.append(((recordIconPositions[i][0] + recordIconPositions[i+1][0]) / 2, (recordIconPositions[i][1] + recordIconPositions[i+1][1]) / 2))

fastIconPositions = list(zip(list1, list2))

# Scale factors
scale_factor_x = 1920 / 2560
scale_factor_y = 1080 / 1440

# Original 2K coordinates
_2kRecordIconPositionsX = [921, 1137, 1137, 921, 921, 1137]
_2kRecordIconPositionsY = [577, 577, 604, 604, 630, 630]

# Calculate 1080p coordinates
_1080pRecordIconPositionsX = [int(x * scale_factor_x) for x in _2kRecordIconPositionsX]
_1080pRecordIconPositionsY = [int(y * scale_factor_y) for y in _2kRecordIconPositionsY]

# Create list of tuples for 1080p coordinates
recordIconPositions1080p = list(zip(_1080pRecordIconPositionsX, _1080pRecordIconPositionsY))

# Calculate extended positions for 1080p
recordIconPositionsExtended1080p = []
for i in range(0, 5):
    recordIconPositionsExtended1080p.append(recordIconPositions1080p[i])
    next_pos = ((recordIconPositions1080p[i][0] + recordIconPositions1080p[i+1][0]) // 2, 
                (recordIconPositions1080p[i][1] + recordIconPositions1080p[i+1][1]) // 2)
    recordIconPositionsExtended1080p.append(next_pos)

### Assembling Playerlist ###

Assembling playerlist from files on the Desktop. Adding players manually is possible here.

In [26]:
import requests
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_uuid_from_username(username):
    """Get UUID for a given username"""
    url = f"https://api.mojang.com/users/profiles/minecraft/{username}"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            profile = response.json()
            return profile['id']
        else:
            return None
    except Exception as e:
        print(f"Error for username {username}: {e}")
        return None

def add_uuids_from_names(names_list, output_file=None, max_workers=10, delay=0.1):
    """Add UUIDs for a list of usernames"""
    
    if isinstance(names_list, str):
        with open(names_list, 'r') as f:
            names = [line.strip() for line in f if line.strip()]
    else:
        names = names_list
    
    print(f"Processing {len(names)} usernames...")
    
    name_to_uuid = {}
    failed_names = []
    
    def fetch_with_delay(username):
        time.sleep(delay)
        uuid = get_uuid_from_username(username)
        return username, uuid
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_name = {executor.submit(fetch_with_delay, name): name for name in names}
        
        for i, future in enumerate(as_completed(future_to_name)):
            username, uuid = future.result()
            if uuid:
                name_to_uuid[username] = uuid
                print(f"Progress: {i+1}/{len(names)} - {username}: {uuid}")
            else:
                failed_names.append(username)
                print(f"Progress: {i+1}/{len(names)} - Failed to get UUID for {username}")
    
    if output_file:
        with open(output_file, 'w') as f:
            json.dump(name_to_uuid, f, indent=2)
        print(f"UUID data saved to: {output_file}")
    
    print(f"\nProcess complete!")
    print(f"Successful: {len(name_to_uuid)}")
    print(f"Failed: {len(failed_names)}")
    
    if failed_names:
        print(f"Failed usernames: {', '.join(failed_names)}")
    
    return name_to_uuid, failed_names

### Add new players ###

Will output UUID:name pairs in player_uuids_fresh.json
-> manually add to player_uuids.json

In [27]:
if __name__ == "__main__":
    # Example usage with a list
    test_names = ["Fflopse"]
    
    # Or from a file:
    # result, failed = add_uuids_from_names('player_names.txt', 'new_player_uuids.json')
    
    result, failed = add_uuids_from_names(test_names, 'player_uuids_fresh.json')
    
    print(f"\nResults:")
    for name, uuid in result.items():
        print(f"  {name}: {uuid}")

Processing 1 usernames...
Progress: 1/1 - Fflopse: 817ead2ca66b4a39aa51a461cef47918
UUID data saved to: player_uuids_fresh.json

Process complete!
Successful: 1
Failed: 0

Results:
  Fflopse: 817ead2ca66b4a39aa51a461cef47918


### get fresh names from mojang api ###

accounts for changed names, as entries here are tied to UUIDs. This part will update the names for all users.

In [28]:
import helpers as h

playerList = h.assemblePlayerList()

Processing 242 players...
Found 242 unique UUIDs
Progress: 1/242 - Nor_Malo
Progress: 2/242 - BusinessBent
Progress: 3/242 - Wissender
Progress: 4/242 - FlyingKyubi
Progress: 5/242 - bawskey
Progress: 6/242 - SatzdesPytag0ras
Progress: 7/242 - G0at3D
Progress: 8/242 - ffawks
Progress: 9/242 - Highsay
Progress: 10/242 - Sackkerl
Progress: 11/242 - JOW23
Progress: 12/242 - cediiiiii_10
Progress: 13/242 - deadfiight
Progress: 14/242 - Wienerisch
Progress: 15/242 - cheesecake2901
Progress: 16/242 - GrafikKatze
Progress: 17/242 - linastunna
Progress: 18/242 - Ungluecklicher
Progress: 19/242 - Lingex
Progress: 20/242 - Teesily
Progress: 21/242 - Gryzes
Progress: 22/242 - Vacted
Progress: 23/242 - camman18YT
Progress: 24/242 - lolisamasan
Progress: 25/242 - TheFabo
Progress: 26/242 - Juti0n
Progress: 27/242 - lizsyy
Progress: 28/242 - VendasTM
Progress: 29/242 - LordAlexos
Progress: 30/242 - SiLoHB
Progress: 31/242 - Simlll
Progress: 32/242 - Flitzi_Dino
Progress: 33/242 - CladyNoClip_
Progre

### old approach to generating playerList (depreciated)

import pyautogui
import os
from tqdm import tqdm, trange

pyautogui.FAILSAFE = True

# check OS 
userOS = os.name

if "nt" in userOS: 
    directory = os.getcwd() + "\\in-new"
elif "posix" in userOS:
    directory = os.getcwd() + "/in-new"
else: 
    directory = None

file_names = []

for file in os.listdir(directory):
    if file.endswith(".txt") and "log" not in file:
        file_names.append(file[:-4])

playerList = sorted(file_names)
renamed = ["BacardiSprite", "jimmi1410timeter", "DerBaumIstDerWeg", "CML_Justin", "DaHinige_", "ryzonabi", "badinfluence666", "officialrzyon", "187leonbande", "Bannbaar", "BearchenGott", "H1ghsay", "Kosmaxnaut", "Livgo", "Lubotter2009", 'JustK1lling', 'schmaarek', "ymq_", "zTotito", "Radiokopf", "toxicplace", "KMaxN", "Liichtenstein"]
for i in renamed:   
    try: playerList.remove(i)
    except: ValueError(i + " not in list.")

newPlayers = ["RasenLP", "FluffyDragon2007", "Tim_Voltia", "BunnyKiko", "BauHD", "Joe3346", "rotmann2", "TAMISH009", "Muniix", "redbrll"]

playerList.extend(newPlayers)

playerLower = [i.lower() for i in playerList]

playerList = sorted(playerList)

playerList = sorted(list(dict.fromkeys(playerList)))

# Check for .txt files in the desktop path and add them to a list, then remove .txt from each string and remove all matching entries from playerList
desktop_path = os.path.expanduser("~/Desktop")
desktop_files = os.listdir(desktop_path)


for file in desktop_files:
    if file.endswith(".txt"):
        file = file[:-4]
    
    try: playerList.remove(file)
    except: pass

# Remove duplicates from playerList
playerList = list(dict.fromkeys(playerList))

# remove "log" and "ItemTooltipExport" from playerList
try: playerList.remove("log")
except: pass
try: playerList.remove("itemTooltipExport")
except: pass


# check for all filenames on the Desktop and remove from playerList (non case sensitive)
for file in desktop_files:
    if file.endswith(".txt"):
        file = file[:-4]
    try: playerList.remove(file)
    except: pass

print(playerList)

### new (just using api-generated name list from UUIDs in player_uuids.json)

cleans playerList: already scraped data will be accounted for -> these playernames will be omitted if record-file is detected on the Desktop

In [29]:
CONSIDER_PROGRESS = True

if CONSIDER_PROGRESS:
    # Check for .txt files in the desktop path and add them to a list, then remove .txt from each string and remove all matching entries from playerList
    desktop_path = os.path.expanduser("~/Desktop")
    desktop_files = os.listdir(desktop_path)


    for file in desktop_files:
        if file.endswith(".txt"):
            file = file[:-4]
        try: playerList.remove(file)
        except: pass

    # Remove duplicates from playerList
    playerList = list(dict.fromkeys(playerList))

    # remove "log" and "ItemTooltipExport" from playerList
    try: playerList.remove("log")
    except: pass
    try: playerList.remove("itemTooltipExport")
    except: pass


    # check for all filenames on the Desktop and remove from playerList (non case sensitive)
    for file in desktop_files:
        if file.endswith(".txt"):
            file = file[:-4]
        try: playerList.remove(file)
        except: pass

print("playerList contains " + str(len((playerList)))+ " unique names")

playerList contains 242 unique names


### Scraping function

adjust resolution parameter or directly adjust coordinates fed to the function depending on what instance / device is being scraped from

In [30]:
import keyboard
import pyautogui
import keyboard
from tqdm import trange


def scrape(resolution = "2k", fast=False, extended=False, dur=None):

    # Scrape Records for Each Player in playerList
    for i, player in enumerate(playerList):

        if sorted(playerList) == sorted(desktop_files): 
            print("All records have been scraped.")
            break

        with trange(len(playerList), desc="Processing players", position=0) as t:
            t.update(i)

            # Open Chat
            pyautogui.press("t")
            pyautogui.sleep(0.2)

            # Assemble Command String
            command = "/rekorde" + " " + player

            # Execute Command
            #pyautogui.typewrite(command, 
            #                    #interval=0.01
            #                    )
            pyautogui.sleep(0.2)
            keyboard.write(command)
            pyautogui.sleep(0.2)

            pyautogui.press("enter")

            pyautogui.sleep(0.15)

            # mooth smoothly through all positions x, y, taking x and y from the respective list

            pyautogui.moveTo(x = 500, y = 500, duration=0.002)

            if resolution == "4k": coordTpls = _4krecordIconPositions
            elif resolution == "2k": coordTpls = recordIconPositions
            elif resolution == "1080p": coordTpls = recordIconPositions1080p
            if extended: coordTpls = recordIconPositionsExtended
            else: coordTpls = recordIconPositions

            if fast or True: 
                coordTpls = fastIconPositions

            duration = 0.7 if not fast else 0.05

            if dur is not None: duration = float(dur)

            if fast or dur is not None: sleep = duration

            for i in range(0, (len(coordTpls))):
                pyautogui.moveTo(x = coordTpls[i], duration=duration)
                # pyautogui.click()
                pyautogui.sleep(sleep)

            pyautogui.press("e")

            # Wait 200 ms until next player] 
            pyautogui.sleep(0.2)

            # Rename file on desktop
            new_filename = player + ".txt"
            desktop_path = os.path.expanduser("~/Desktop")
            old_filepath = os.path.join(desktop_path, "ItemTooltipExport.txt")
            new_filepath = os.path.join(desktop_path, new_filename)
            try: os.rename(old_filepath, new_filepath)
            except: print(f"Error renaming file for {player}.")

            # if log: print(f"Record for {player} <{i+1}> scraped successfully.\n")

def verifyDesktop(debug=False):
    # verify the linecount for each file, if any file has more or less than 21 lines, print the filename, else give an OK statement, if there is no file for a player, also point that out. List all players in each "category" at once, not as individual line
    for player in playerList:
        c = 0
        try:
            with open(os.path.join(desktop_path, player + ".txt"), "r") as file:
                try: 
                    lines = file.readlines()
                    if debug: print(player + " has " + str(len(lines)) + " lines.")
                    elif len(lines) != 24:
                        print(player + " has " + str(len(lines)) + " lines.")
                        c += 1
                except: pass
        except FileNotFoundError:
            print(player + " has no file.")
    print(f"All files checked. {c} files had an unexpected line count.")

def clear():
    try: os.remove(os.path.join(desktop_path, "ItemTooltipExport.txt"))
    except: pass
    try: os.remove(os.path.join(desktop_path, "log.txt"))
    except: pass

In [31]:
# clear()

pyautogui.sleep(5)

scrape(fast=False, dur=0.002, resolution="2k")

pyautogui.sleep(2)

clear()

verifyDesktop()

Processing players: 100%|█████████▉| 241/242 [00:04<00:00, 55.33it/s]


Dokkkkko has 22 lines.
Falke_01 has 22 lines.
GeneralEnte06 has 23 lines.
All files checked. 0 files had an unexpected line count.
